In [38]:
import dpkt
import struct
import typing
import argparse
import math
import matplotlib.pyplot as plt
import numpy as np
import sys
import time
from tqdm import tqdm


def get_udp_payload_bytes(pcap_filename) -> typing.List[bytes]:
    """
    Read UDP payload bytes from a .pcap(ng) file
    return list with one entry per packet, entry containing udp payload
    """
    timestamps = []
    try:
        with open(pcap_filename, "rb") as f:
            pcap_read = dpkt.pcap.UniversalReader(f)
            udp_payloads = list()
            for ts, buf in pcap_read:
                # skip packet if not long enough to contain IP+UDP+CODIF hdrs
                if len(buf) < (34 + 8 + 64):
                    print(f"WARNING: Found packet that is too small {len(buf)}Bytes")
                    continue
                eth = dpkt.ethernet.Ethernet(buf)
                ip = eth.data
                # skip non-UDP packets
                if ip.p != dpkt.ip.IP_PROTO_UDP:
                    print(f"WARNING: Found packet that is not UDP {ip.p} type")
                    continue
                # add the UDP payload data into the list of payloads
                udp = ip.data
                udp_payloads.append(udp.data)
                timestamps.append(ts)
    except FileNotFoundError as fnf_err:
        print(fnf_err)
        sys.exit(1)

    return timestamps, udp_payloads


N_POL = 2
N_VALS_PER_CPLX = 2
N_BYES_PER_VAL = 1
N_BYTES_PER_SAMPLE = N_POL * N_VALS_PER_CPLX * N_BYES_PER_VAL


def parse_args():
    parser = argparse.ArgumentParser(prog="pss packet capture analyser")
    parser.add_argument("-f", "--file", required=True, help="File to analyse")
    parser.add_argument("-t", "--txt", type=str, help="Title text")
    return parser.parse_args()



#arg_parser = parse_args()

lambda_file = "/Users/jsmallwood/Documents/projects/cuda-spatial-filtering/scripts/cap_13Dec2024_0.pcapng"
total_ADCs = 10
pcount_max = 900

# read pcap file
tstamps, payloads = get_udp_payload_bytes(lambda_file)

# run through the data to find the number of beams and channels
first_pkt = True
start_seq_no = 0
start_chan = 0
end_seq_no = 0
end_chan = 0
total_packets = 0

for ts, pkt_payload in zip(tstamps, payloads):
    seq_no = struct.unpack("<Q", pkt_payload[0:8])[0]
    FPGA_id = struct.unpack("<I", pkt_payload[8:12])[0]
    freq_chan = struct.unpack("<H", pkt_payload[12:14])[0]
    total_packets += 1
    if first_pkt:
        first_pkt = False
        start_seq_no = seq_no
        end_seq_no = seq_no
        start_chan = freq_chan
        end_chan = freq_chan
    else:
        if freq_chan < start_chan:
            start_chan = freq_chan
        if seq_no < start_seq_no:
            start_seq_no = seq_no
        if freq_chan > end_chan:
            end_chan = freq_chan
        if seq_no > end_seq_no:
            end_seq_no = seq_no

print(f"Found {total_packets} packets")
print(
    f"Start time sample = {start_seq_no}, total time samples = {end_seq_no - start_seq_no + 1}, (= total time {1080e-9 * (end_seq_no - start_seq_no + 1)} seconds)"
)
print(
    f"Start channel = {start_chan}, total channels = {(end_chan - start_chan) + 1}"
)
total_channels = (end_chan - start_chan) + 1
total_time_packets = (end_seq_no - start_seq_no + 1) // 64
expected_packets = total_channels * total_time_packets
print(f"expected packets = {expected_packets}")

# Get all the data into a big numpy array
# ADCs x channels x time samples
all_samples = np.zeros(
    (total_ADCs, total_channels, (end_seq_no - start_seq_no + 64), N_POL),
    dtype=np.complex64,
)
all_samples_scaled = np.zeros(
    (total_ADCs, total_channels, (end_seq_no - start_seq_no + 64), N_POL),
    dtype=np.complex64,
)
# scale factor for each sample, initialise with -1
all_scales = -500 * np.ones(
    (total_ADCs, total_channels, (end_seq_no - start_seq_no + 64), N_POL), dtype=np.float32
)
pkt_scale = np.zeros((total_ADCs, N_POL), dtype=np.float32)
pcount = 0
for ts, pkt_payload in tqdm(zip(tstamps, payloads), total=expected_packets):
    pcount += 1
    seq_no = struct.unpack("<Q", pkt_payload[0:8])[0]
    FPGA_id = struct.unpack("<I", pkt_payload[8:12])[0]
    freq_chan = struct.unpack("<H", pkt_payload[12:14])[0] - start_chan
    padding = struct.unpack("<Q", pkt_payload[14:22])[0]

    # Get scale factors
    for adc in range(total_ADCs):
        for p in range(N_POL):
            pkt_scale[adc][p] = 1#np.float32(
            #struct.unpack("H", pkt_payload[(22 + 2 * adc * N_POL + 2 * p) : (24 + 2 * adc * N_POL + 2 * p)])[0]
        #)
    # Get data
    data_base = 22 + total_ADCs * 2 * N_POL
    for adc in range(total_ADCs):
        for p in range(N_POL):
            for t in range(64):  # 64 time samples per packet
                x_i, x_q = struct.unpack(
                    "bb",
                    pkt_payload[
                        (data_base + t * total_ADCs * N_POL * 2 + adc * 2 * N_POL + 2 * p) : (
                            data_base + t * total_ADCs *N_POL * 2 + adc * 2 * N_POL + 2 * p + 2
                        )
                    ],
                )
                all_samples[adc, freq_chan, seq_no - start_seq_no + t, p] = (
                    1j * np.float32(x_q) + np.float32(x_i)
                )
                all_samples_scaled[adc, freq_chan, seq_no - start_seq_no + t,p] = (
                    pkt_scale[adc][p] * (1j * np.float32(x_q) + np.float32(x_i))
                )
                all_scales[adc, freq_chan, seq_no - start_seq_no + t,p] = pkt_scale[adc][p]
    #if pcount >= pcount_max:
    #    print(f"stopping packet decoding at packet {pcount}")
    #    break

# plt_chan = 0
# for adc_base in range(2):
#     plt.figure()
#     plt.grid(True)
#     for adc in range(5):
#         for p in range(N_POL):
#             plt.subplot(5, 2, 2 * adc + p + 1)
#             plt.plot(
#                 np.real(all_samples_scaled[adc_base * 5 + adc, plt_chan, 0:2048,p]), "r.-"
#             )
#             plt.plot(
#                 np.imag(all_samples_scaled[adc_base * 5 + adc, plt_chan, 0:2048,p]), "g.-"
#             )
#             plt.plot(all_scales[adc_base * 5 + adc, plt_chan, 0:512,p], "b-")
#             plt.title(f"Complex samples for adc {adc_base * 5 + adc} pol {p}, scale blue")

    
#     plt.show()



Found 103143 packets
Start time sample = 55131776, total time samples = 825089, (= total time 0.89109612 seconds)
Start channel = 252, total channels = 8
expected packets = 103136


103143it [07:41, 223.47it/s]                                                                                                                                                                                    


In [39]:
import numpy as np

def triangular_adc_pairs(N):
    """Return list of (i, j) index pairs for lower-triangle storage."""
    pairs = []
    for i in range(N):        # row
        for j in range(i+1):  # col
            pairs.append((i, j))
    return pairs

def hermitian_from_lower_triangular(vec, n):
    """
    vec: 1D array of length n(n+1)/2 containing the LOWER triangle row-by-row.
    n: size of the Hermitian matrix.
    """
    H = np.zeros((n, n), dtype=complex)

    # Fill lower triangle
    idx = 0
    for i in range(n):
        for j in range(i + 1):
            H[i, j] = vec[idx]
            idx += 1

    # Copy lower to upper (conjugate)
    H = H + np.tril(H, -1).conj().T
    return H

In [40]:
corr_mat = np.zeros(
    (total_channels, int(total_ADCs * (total_ADCs + 1 ) /2), N_POL, N_POL),
    dtype=np.complex64,
)
pairs = triangular_adc_pairs(total_ADCs)

for f in range(total_channels): 
    S = all_samples_scaled[:, f, :, :]   # shape: (ADCs, time, pol)

    for t_idx, (i, j) in enumerate(pairs):
        # S[i] = shape (time, pol)
        # Outer-product over polarization:
        #
        #    Corr[p,q] = sum_t S[i,t,p] * conj(S[j,t,q])
        #
        corr_mat[f, t_idx] = S[i].conj().T @ S[j]
                

In [45]:
corr_mat.shape

(8, 55, 2, 2)

In [28]:
np.save("cap_0.np", all_samples_scaled)

In [53]:
all_samples[0,0,:,0] @ all_samples[0,0,:,0].conj().T

np.complex64(8883+0j)

In [33]:
all_samples_scaled.shape

(10, 8, 825152, 2)

In [43]:
all_scales

array([[[[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        ...,

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]]],


       [[[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
   